<a href="https://colab.research.google.com/github/Rincodecpp/Rincodecpp/blob/main/Mask2Former_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

# --extra-index-url https://download.pytorch.org/whl
# torch==1.10.1+cu111
# torchvision==0.11.2+cu111
# torchaudio==0.10.1
# -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html

# detectron2==0.6
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
# pip install detectron2 -f \
# https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html

!pip install gradio opencv-python scipy

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-nk_xwoub
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-nk_xwoub
  Resolved https://github.com/facebookresearch/detectron2.git to commit 754469e176b224d17460612bdaa2cb8112b04cd9
  Preparing metadata (setup.py) ... done
  Using cached gradio-5.6.0-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached fastapi-0.115.5-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.4.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio_client-1.4.3-py3-none-any.whl.metadata (7.1 kB)
  Using cached MarkupSafe-2.1.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0

In [2]:
import torch, detectron2

!nvcc --version

TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch:", TORCH_VERSION, "; cuda:", CUDA_VERSION)

# print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch: 2.5 ; cuda: cu121


In [3]:
# !pip install transformers  # latest stuff not included yet!
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [9]:
def coco_panoptic_palette():
      return [ (220, 20, 60), (119, 11, 32), (0, 0, 142), (0, 0, 230), (106, 0, 228), (0, 60, 100), (0, 80, 100), (0, 0, 70), (0, 0, 192), (250, 170, 30), (100, 170, 30), (220, 220, 0), (175, 116, 175), (250, 0, 30), (165, 42, 42), (255, 77, 255), (0, 226, 252), (182, 182, 255), (0, 82, 0), (120, 166, 157), (110, 76, 0), (174, 57, 255), (199, 100, 0), (72, 0, 118), (255, 179, 240), (0, 125, 92), (209, 0, 151), (188, 208, 182), (0, 220, 176), (255, 99, 164), (92, 0, 73), (133, 129, 255), (78, 180, 255), (0, 228, 0), (174, 255, 243), (45, 89, 255), (134, 134, 183), (145, 148, 174), (255, 208, 186),(197, 226, 255), (171, 134, 1), (109, 63, 54), (207, 138, 255), (151, 0, 95),(9, 80, 61), (84, 105, 51), (74, 65, 105), (166, 196, 102), (288, 195, 210), (255, 109, 65), (6, 143, 149), (179, 0, 194), (209, 99, 106), (5, 121, 0), (227, 255, 205), (147, 186, 208), (153, 69, 1), (3, 95, 161), (163, 255, 8), (119, 8, 170), (0, 182, 199), (8, 165, 120), (183, 130, 88), (95, 32, 8), (138, 114, 135), (110, 129, 133), (166, 74, 118), (219, 142, 185), (79, 210, 114), (178, 90, 62), (65, 70, 15), (127, 167, 115), (59, 105, 106), (142, 188, 45), (196, 172, 0), (95, 54, 80), (128, 76, 255), (201, 57, 1), (246, 8, 122), (191, 162, 208), (255, 255, 128), (147, 211, 203), (150, 100, 100), (168, 171, 172), (146, 112, 198), (216, 170, 100), (92, 136, 89), (218, 88, 184), (241, 129, 0), (217, 17, 255), (124, 74, 181), (70, 70, 70), (255, 228, 255), (154, 288, 0), (193, 8, 92), (76, 91, 113), (255, 180, 195), (106, 154, 176), (230, 150, 140), (60, 143, 255), (128, 64, 128), (92, 82, 55), (254, 212, 124), (73, 77, 174), (255, 160, 98), (255, 255, 255), (104, 84, 109), (169, 164, 131), (225, 199, 255), (137, 54, 74), (135, 158, 223), (7, 246, 231), (107, 255, 200), (58, 41, 149), (183, 121, 142), (255, 73, 97), (107, 142, 35), (190, 153, 153), (146, 139, 141), (70, 130, 180), (134, 199, 156), (209, 226, 140), (96, 36, 108), (96, 96, 96), (64, 170, 64), (152, 251, 152), (208, 229, 228), (206, 186, 171), (152, 161, 64), (116, 112, 0), (0, 114, 143), (102, 102, 156), (250, 141, 255)]

def cityscapes_palette():
      return [[128, 64, 128], [244, 35, 232],[70, 70, 70],[102, 182, 156],[190, 153, 153], [153, 153, 153],[250, 170, 30],[220, 220, 0],[107, 142, 35],[152, 251, 152], [70, 130, 180], [220, 20, 60], [255, 0, 0], [0, 0, 142], [0, 0, 70], [0, 60, 100], [0, 80, 100], [0, 0, 230], [119, 11, 32]]

def ade_palette():
    """Color palette that maps each class to RGB values.
    This one is actually taken from ADE20k.
    """
    return [[120, 120, 120], [180, 120, 120], [6, 230, 230], [80, 50, 50], [4, 200, 3], [120, 120, 80], [140, 140, 140], [204, 5, 255], [230, 230, 230], [4, 250, 7], [224, 5, 255], [235, 255, 7], [150, 5, 61], [120, 120, 70], [8, 255, 51], [255, 6, 82], [143, 255, 140], [284, 255, 4], [255, 51, 7], [284, 78, 3], [0, 102, 200], [61, 230, 250], [255, 6, 51], [11, 102, 255], [255, 7, 71], [255, 9, 224], [9, 7, 230], [220, 220, 220], [255, 9, 92], [112, 9, 255], [8, 255, 214], [7, 255, 224], [255, 184, 6], [10, 255, 71], [255, 41, 10], [7, 255, 255], [224, 255, 8], [102, 8, 255], [255, 61, 6], [255, 194, 7], [255, 122, 8], [0, 255, 20], [255, 8, 41], [255, 5, 153], [6, 51, 255], [235, 12, 255], [160, 150, 20], [0, 163, 255], [140, 140, 140], [250, 10, 15], [20, 255, 0], [31, 255, 0], [255, 31, 0], [255, 224, 0], [153, 255, 0], [0, 0, 255], [255, 71, 0], [0, 235, 255], [0, 173, 255], [31, 0, 255], [11, 200, 200], [255, 82, 0], [0, 255, 245], [0, 61, 255], [0, 255, 112], [0, 255, 133], [255, 0, 0], [255, 163, 0], [255, 102, 0], [194, 255, 0], [0, 143, 255], [51, 255, 0], [0, 82, 255], [0, 255, 41], [0, 255, 173], [10, 0, 255], [173, 255, 0], [0, 255, 153], [255, 92, 0], [255, 0, 255], [255, 0, 245], [255, 0, 102], [255, 173, 0], [255, 184, 184], [0, 31, 255], [0, 255, 61], [255, 0, 20], [0, 71, 255], [255, 0, 204], [0, 255, 194], [0, 255, 82], [0, 10, 255], [0, 112, 255], [51, 0, 255], [0, 194, 255], [0, 122, 255], [0, 255, 163], [255, 153, 0], [0, 255, 10], [255, 112, 0], [143, 255, 0], [82, 0, 255], [163, 255, 0], [255, 235, 0], [8, 184, 170], [133, 0, 255], [0, 255, 92], [184, 0, 255], [255, 0, 31], [0, 184, 255], [0, 214, 255], [255, 0, 112], [92, 255, 0], [0, 224, 255], [112, 224, 255], [70, 184, 160], [163, 0, 255], [153, 0, 255], [71, 255, 0], [255, 0, 163], [255, 204, 0], [255, 0, 143], [0, 255, 235], [133, 255, 0], [255, 0, 235], [245, 0, 255], [255, 0, 122], [255, 245, 0], [10, 190, 212], [214, 255, 0], [0, 204, 255], [20, 0, 255], [255, 255, 0], [0, 153, 255], [0, 41, 255], [0, 255, 204], [41, 0, 255], [41, 255, 0], [173, 0, 255], [0, 245, 255], [71, 0, 255], [122, 0, 255], [0, 255, 184], [0, 92, 255], [184, 255, 0], [0, 133, 255], [255, 214, 0], [25, 194, 194], [102, 255, 0], [92, 0, 255]]

In [12]:
import torch
import random
import numpy as np
from PIL import Image
from collections import defaultdict
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import ColorMode, Visualizer
from transformers import MaskFormerImageProcessor, Mask2FormerForUniversalSegmentation

In [14]:
from transformers import Mask2FormerConfig, Mask2FormerModel
# Initializing a Mask2Former facebook/mask2former-swin-small-coco-instance configuration
configuration = Mask2FormerConfig()
# Initializing a model (with random weights) from the facebook/mask2former-swin-small-coco-ins
model = Mask2FormerModel (configuration)
# Accessing the model configuration
configuration = model.config
configuration

Mask2FormerConfig {
  "_attn_implementation_autoset": true,
  "activation_function": "relu",
  "backbone": null,
  "backbone_config": {
    "depths": [
      2,
      2,
      18,
      2
    ],
    "drop_path_rate": 0.3,
    "in_channels": 3,
    "model_type": "swin",
    "out_features": [
      "stage1",
      "stage2",
      "stage3",
      "stage4"
    ],
    "out_indices": [
      1,
      2,
      3,
      4
    ]
  },
  "backbone_kwargs": null,
  "class_weight": 2.0,
  "common_stride": 4,
  "decoder_layers": 10,
  "dice_weight": 5.0,
  "dim_feedforward": 2048,
  "dropout": 0.0,
  "encoder_feedforward_dim": 1024,
  "encoder_layers": 6,
  "enforce_input_projection": false,
  "feature_size": 256,
  "feature_strides": [
    4,
    8,
    16,
    32
  ],
  "hidden_dim": 256,
  "ignore_value": 255,
  "importance_sample_ratio": 0.75,
  "init_std": 0.02,
  "init_xavier_std": 1.0,
  "mask_feature_size": 256,
  "mask_weight": 5.0,
  "model_type": "mask2former",
  "no_object_weight": 0.1,


In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
def load_model_and_processor (model_ckpt: str, device:str):
  model = Mask2Former
  model.eval()
  ForUniversalSegmentation.from_pretrained(model_ckpt).to(torch.device (device))
  image_preprocessor = MaskFormerImageProcessor.from_pretrained (model_ckpt)
  return model, image_preprocessor
# https://huggingface.co/facebook/mask2former-swin-base-IN21k-ade-semantic

# Choose what model to download from Huggingface for SEMANTIC or PANOPTIC (everythink visible) ta
def load_default_ckpt(segmentation_task: str):
  if segmentation_task == "semantic":
    default_pretrained_ckpt = "facebook/mask2former-swin-base-ade-semantic" #facebook/mask2fo
  elif segmentation_task == "panoptic":
    default_pretrained_ckpt = "facebook/mask2former-swin-large-coco-panoptic" # facebook/mask
  return default_pretrained_ckpt

def draw_panoptic_segmentation (predicted_segmentation_map, seg_info, image):
    metadata = MetadataCatalog.get("coco_2017_val_panoptic")
    for res in seg_info:
        res['category_id'] = res.pop('label_id')
        pred_class = res['category_id']
        isthing = pred_class in metadata.thing_dataset_id_to_contiguous_id.values()
        res['isthing'] = bool(isthing)


        visualizer = Visualizer(np.array(image)[:, :, ::-1], metadata=metadata, instance_mode=ColorMode. IMAGE)
        out=visualizer.draw_panoptic_seg_predictions(
        predicted_segmentation_map.cpu(), seg_info, alpha=0.5
        )
        output_img = Image.fromarray(out.get_image())
    return output_img


def draw_semantic_segmentation (segmentation_map, image, palette):
    color_segmentation_map = np.zeros((segmentation_map.shape[0], segmentation_map.shape[1], 3), dtype=np. uint8)
    for label, color in enumerate(palette):
        color_segmentation_map[segmentation_map - 1 == label, :] = color
    # Convert to BGR
    ground_truth_color_seg = color_segmentation_map[..., ::-1]

    img = np.array(image) * 0.5 + ground_truth_color_seg * 0.5
    img = img.astype(np.uint8)
    return img

# Main sequence of code:
def predict_masks(input_img_path: str, segmentation_task: str):

    #load model checkpoints and image processor for it:
    default_pretrained_ckpt = load_default_ckpt(segmentation_task)
    model, image_processor = load_model_and_processor (default_pretrained_ckpt, device)

    ## pass input image through image processor
    image = Image.open(input_img_path)
    inputs = image_processor(images=image, return_tensors="pt").to(torch.device(device))

    ## pass inputs to model for prediction
    with torch.no_grad():
        outputs = model(**inputs)


    # pass outputs to processor for POSTPROCESSING:
    if segmentation_task == "semantic":
        result = image_processor.post_process_semantic_segmentation(outputs, target_sizes=[image.size[::-1]])[0]
        predicted_segmentation_map = result.cpu().numpy()
        palette = ade_palette()
        output_result = draw_semantic_segmentation (predicted_segmentation_map, image, palette)
    else:
        result = image_processor.post_process_panoptic_segmentation(outputs, target_sizes=[image.size[::-1]])[0]
        predicted_segmentation_map = result["segmentation"]
        seg_info = result['segments_info']
        output_result = draw_panoptic_segmentation(predicted_segmentation_map, seg_info, image)
    return output_result

In [32]:
import gradio as gr

with gr.Blocks() as Mask2former_GUI:

    gr.Markdown("# **<p align='center'>Mask2Former for Panoptic and Semantic Segmentation</p>**")
    gr.Markdown("<p align='center'> ·A· Mask2Former GUI to run on a free COLAB (CPU and/or GPU)</p>")

    with gr.Group():
        with gr.Row():
            choose_segmentation = gr.Dropdown(["semantic", "panoptic"], value="panoptic", label="Panoptic or Semantic Segmentation")
        with gr.Group():
            with gr.Row():
                with gr.Column(scale=1):
                    input_image = gr.Image(type='filepath', label="Place your image here:", show_label=True)
                with gr.Column(scale=3):
                    output_mask = gr.Image(label="Mask2Former Output:", show_label=True)

    with gr.Group():
        with gr.Row():
            with gr.Column(scale=1):
                start_run = gr.Button("Run Mask2Former-Inference")
            with gr.Column(scale=2):
                gr.Markdown("""
                My Mask2Former models are:
                - facebook/mask2former-swin-base-ade-semantic
                - facebook/mask2former-swin-large-coco-panoptic
                """)

    start_run.click(predict_masks, inputs=[input_image, choose_segmentation], outputs=output_mask)

if __name__ == "__main__":
    Mask2former_GUI.launch(share=True, debug=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://91ff6c814b7e75c583.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
